# FastAPI 개념 (2)


In [ ]:
!pip install fastapi uvicorn pyngrok nest_asyncio aiofiles python-multipart

In [ ]:
!pip install aiofiles

In [ ]:
%%writefile test.txt

'Hello, World!'

Overwriting test.txt


#### Templates을 사용하는 이유❓
- 매번 텍스트로 작성할 수 도있지만, 번거롭고 예쁘게 디자인하기 힘들다 
- 형식을 만들어 놓고 재활용 할 수 있도록 templates를 구성한다 
- templates 기반으로 **치환**이 되는 부분, `{key : value}` 값이 입력됨 

In [ ]:
%%writefile templates/item.html

<html>
<head>
    <title>Item Details</title>
    <link href="{{ url_for('static', path='/styles.css') }}" rel="stylesheet">
</head>
<body>
    <h1>Item ID: {{ id }}</h1>
</body>
</html>

Writing templates/item.html


In [ ]:
%%writefile static/styles.css
h1 {
    color: green;
}

Writing static/styles.css


In [ ]:
from fastapi import FastAPI, File, UploadFile
from typing import List
import os

import nest_asyncio
from pyngrok import ngrok
import uvicorn

* File Upload & Downloads

In [ ]:
# 에러 발생시 코드 확인 
!ps

# 에러 유발 코드 삭제 
!kill -9 292

    PID TTY          TIME CMD
      1 ?        00:00:06 node
     17 ?        00:00:00 tail
     53 ?        00:00:07 jupyter-noteboo
     54 ?        00:00:04 dap_multiplexer
     65 ?        00:00:23 python3
     85 ?        00:00:09 python3
    723 ?        00:00:00 ps


In [ ]:
from fastapi import FastAPI, Request
from fastapi import File, UploadFile
from fastapi.responses import FileResponse
from fastapi.responses import HTMLResponse
from fastapi.staticfiles import StaticFiles
from fastapi.templating import Jinja2Templates
from typing import List
import os
import nest_asyncio
from pyngrok import ngrok
import uvicorn

app = FastAPI()

# get file 
@app.get('/getfile/{fn}')
async def getFile(fn:str):
  file_path = os.getcwd() + '/' + fn
  return FileResponse(path = file_path, media_type='application/octet-stream', filename=fn)

# file upload
@app.post("/uploadfiles")
async def create_upload_files(files: List[UploadFile] = File(...)):
    UPLOAD_DIRECTORY = "./"
    for file in files:
        contents = await file.read()
        with open(os.path.join(UPLOAD_DIRECTORY, file.filename), "wb") as fp:
            fp.write(contents)
        print(file.filename)
    return {"filenames": [file.filename for file in files]}

# Using Templates (Jinja2)
app.mount("/static", StaticFiles(directory="static"), name="static")
templates = Jinja2Templates(directory="templates")

@app.get("/items/{id}", response_class=HTMLResponse)
async def read_item(request: Request, id: str):
    return templates.TemplateResponse("item.html", {"request": request, "id": id})


ngrok_tunnel = ngrok.connect(8000)
print ('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, host='0.0.0.0', port=8000)

Public URL: http://0ea9-35-247-35-82.ngrok.io


INFO:     Started server process [2286]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     114.203.149.236:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     114.203.149.236:0 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     114.203.149.236:0 - "GET /items/test HTTP/1.1" 200 OK
INFO:     114.203.149.236:0 - "GET /items/test HTTP/1.1" 200 OK
INFO:     114.203.149.236:0 - "GET /static/styles.css HTTP/1.1" 200 OK
INFO:     114.203.149.236:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [2286]
